In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

years = list(range(2011, 2025))
all_data = []

for year in tqdm(years):
    url = f"https://www.baseball-reference.com/leagues/MLB/{year}-standings.shtml"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the standings tables
    tables = soup.find_all("table")

    for table in tables:
        # Usually standings tables have an id starting with "standings_"
        if "standings" in table.get("id", ""):
            df = pd.read_html(str(table))[0]

            # The dataframe might contain division titles (AL East, NL West etc.) — handle that
            if 'W' in df.columns:
                df['Year'] = year
                all_data.append(df)

# Combine all dataframes
records_df = pd.concat(all_data, ignore_index=True)

# Clean up
records_df = records_df.rename(columns={
    'Tm': 'Team',
    'W': 'Wins',
    'L': 'Losses',
    'W-L%': 'Win_Percentage'
})

# Drop rows where 'Team' is NaN (division headers, etc.)
records_df = records_df[records_df['Team'].notna()]

print(records_df.head())


  0%|          | 0/14 [00:00<?, ?it/s]/tmp/ipykernel_16234/2209702857.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
/tmp/ipykernel_16234/2209702857.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
/tmp/ipykernel_16234/2209702857.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
/tmp/ipykernel_16234/2209702857.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(t

                Team  Wins  Losses  Win_Percentage    GB  Year
0   New York Yankees    97      65           0.599    --  2011
1     Tampa Bay Rays    91      71           0.562   6.0  2011
2     Boston Red Sox    90      72           0.556   7.0  2011
3  Toronto Blue Jays    81      81           0.500  16.0  2011
4  Baltimore Orioles    69      93           0.426  28.0  2011


In [3]:
records_df.to_csv("mlb_team_records_2011_2024.csv", index=False)
print("✅ CSV file saved as mlb_team_records_2011_2024.csv")


✅ CSV file saved as mlb_team_records_2011_2024.csv
